In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from config import MINIO_BUCKET_RAW, MINIO_BUCKET_PROCESSED
from etl.transformer import transform_html_to_tables, transform_tables
from etl.writer import write_to_parquet, write_to_postgres
from etl.reader import read_htmls_from_minio, read_from_parquet, read_from_postgres
from clients.postgres_client import get_postgres_properties
from clients.spark_client import create_spark_session
from logger import get_logger
import pyspark.sql.functions as F




In [ ]:
spark = create_spark_session()
postgres_props = get_postgres_properties()



2025-05-04 15:27:24,724 - INFO - VRNFSO_ETL - Starting ETL job


In [ ]:
html_pairs = read_htmls_from_minio(MINIO_BUCKET_RAW)

events_df, distances_df, results_df = transform_html_to_tables(html_pairs, spark)

2025-05-04 15:27:33,126 - INFO - VRNFSO_ETL - Читаем HTML из MinIO
2025-05-04 15:27:33,459 - INFO - VRNFSO_ETL - Парсим HTML в DataFrame


In [ ]:

write_to_parquet(events_df, f"s3a://{MINIO_BUCKET_PROCESSED}/events/")
write_to_parquet(distances_df, f"s3a://{MINIO_BUCKET_PROCESSED}/distances/")
write_to_parquet(results_df, f"s3a://{MINIO_BUCKET_PROCESSED}/results/")

25/05/02 21:31:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/02 21:32:01 WARN DAGScheduler: Broadcasting large task binary with size 1004.1 KiB
25/05/02 21:32:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/05/02 21:32:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
logger.info("Считываем из Parquet")
read_events_df = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/events/")
read_distances_df = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/distances/")
read_results_df = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/results/")

25/05/04 11:56:06 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [ ]:
transformed_events, transformed_groups, transformed_participants, transformed_results = transform_tables(read_events_df, read_distances_df, read_results_df)

In [10]:
write_to_parquet(transformed_events, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_events/")
write_to_parquet(transformed_groups, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_groups/")
write_to_parquet(transformed_participants, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_participants/")
write_to_parquet(transformed_results, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_results/")

25/05/04 11:59:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [39]:
read_transformed_events = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_events/")
read_transformed_groups = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_groups/")
read_transformed_participants = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_participants/")
read_transformed_results = read_from_parquet(spark, f"s3a://{MINIO_BUCKET_PROCESSED}/transformed_results/")

In [15]:
write_to_postgres(read_transformed_events, "events", postgres_props)


In [40]:
events_lookup = read_from_postgres(spark, "events", postgres_props).select("event_id", "event_date")


In [41]:
read_transformed_groups = read_transformed_groups.join(
    events_lookup,
    on="event_date",
    how="inner"
).select("group_name", "cp", "length_km", "event_id")

In [43]:
read_transformed_groups.show()

+----------+---+---------+--------+
|group_name| cp|length_km|event_id|
+----------+---+---------+--------+
|   Женщины|  8|      4.1|       1|
|   Мужчины| 10|      5.4|       1|
|       Ж16| 16|      2.3|       3|
|       М18| 17|      2.9|       3|
|       Ж55| 16|      2.3|       3|
|       Ж12|  8|      1.3|       3|
|       М35| 17|      2.9|       3|
|       М55| 14|      2.2|       3|
|       М12| 10|      1.5|       3|
|   Новички|  9|      1.1|       3|
|       Ж35| 15|      2.4|       3|
|       Ж14|  9|      1.4|       3|
|        ЖЭ| 17|      2.9|       3|
|       Ж18| 15|      2.4|       3|
|       Ж10|  8|      1.3|       3|
|       М14|  9|      1.6|       3|
|       М16| 14|      2.2|       3|
|       М10|  8|      1.3|       3|
|        МЭ| 17|      3.2|       3|
|        ЖЭ| 13|      2.4|       4|
+----------+---+---------+--------+
only showing top 20 rows



In [19]:
write_to_postgres(read_transformed_groups, "group_params", postgres_props)


In [44]:
groups_lookup = read_from_postgres(spark, "group_params", postgres_props).select("group_id", "event_id", "group_name")

In [ ]:
read_transformed_participants = read_transformed_participants.groupBy("full_name", "birth_year").agg(F.collect_set("team").alias("team"))

In [52]:
write_to_postgres(read_transformed_participants, "participants", postgres_props)


In [53]:
participants_lookup = read_from_postgres(spark, "participants", postgres_props).select("participant_id", "full_name", "team", "birth_year")

In [54]:
read_transformed_results.show()

+----------+----------+---------------+-----------------+-------------------+-------------+----------+----------+---------------+-----------+--------+
|event_date|group_name|position_number|        full_name|               team|qualification|bib_number|birth_year|finish_position|result_time|time_gap|
+----------+----------+---------------+-----------------+-------------------+-------------+----------+----------+---------------+-----------+--------+
|2024-05-18|      М-21|             24|    Ершов Дмитрий|СШОР 18 Вильденберг|            I|      1238|      2005|             24|   00:45:36|  +11:34|
|2024-05-18|      М-21|             25|   Пономарёв Иван|        Жидкий Хлеб|           МС|      1185|      1996|             25|   00:45:50|  +11:48|
|2024-05-18|      М-21|             26|   Бунегин Кирилл|      СШ 18 ИМПУЛЬС|          КМС|      1224|      2004|             26|   00:46:47|  +12:45|
|2024-05-18|      М-21|             27|     Фролов Роман|      СШОР 18 Олимп|            I|   

In [56]:
read_transformed_results = read_transformed_results \
    .join(events_lookup, on="event_date", how="inner") \
    .join(groups_lookup, on=["event_id","group_name"], how="inner") \
    .join(participants_lookup, on=["full_name","birth_year"], how="inner") \
    .select(
        "event_id",
        "group_id",
        "participant_id",
        "position_number",
        "qualification",
        "bib_number",
        "finish_position",
        "result_time",
        "time_gap"
    )

In [57]:
write_to_postgres(read_transformed_results, "results", postgres_props)